*KNOWLEDGE BASE EXAMPLES*
===
- [0.PRINT TRICKS](#0.PRINT-TRICKS)
- [1.PREPROCESSING](#1.PREPROCESSING)
    - [1.1 ONE HOT ENCODING](#1.1-ONE-HOT-ENCODING)
    - [1.2 NORMALIZATION](#1.2-NORMALIZATION)
    - [1.3 PCA](#1.3-PCA)
    - [1.4 FEATURE IMPORTANCE](#1.4-FEATURE-IMPORTANCE)
- [2.FILE MANAGEMENT](#2.FILE-MANAGEMEMT)
- [3.PROCESS AND MEMORY MANAGEMENT](3.#PROCESS-AND-MEMORY-MANAGEMENT)
- [4.DATA SHAPE MANAGEMENT](#4.DATA-SHAPE-MANAGEMENT)
- [5.CORRELATION HEATMAPS](#5.CORRELATION-HEATMAPS)
- [6.STRING MANAGEMENT](#6.STRING-MANAGEMENT)
- [7.PLOT FUNCTIONS](#7.PLOT-FUNCTIONS)

0.PRINT TRICKS
===

In [ ]:
#Summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
print("Variable value: {}".format(variable))

1.PREPROCESSING
===

1.1 ONE HOT ENCODING
--

In [ ]:
#ONE HOT ENCODING WITH OneHoteEncoder
categorical_cols = ['Pclass', 'Sex' , 'CabinDeck', 'Embarked', 'Title', 'TicketType']
labeled_df = df_data_6[categorical_cols]
enc = preprocessing.OneHotEncoder()
enc.fit(labeled_df)

onehotlabels = enc.transform(labeled_df).toarray()

new_columns=list()
for col, values in zip(labeled_df.columns, enc.categories_):
    new_columns.extend([col + '_' + str(value) for value in values])

df_data_8= pd.concat([df_data_6, pd.DataFrame(onehotlabels, columns=new_columns)], axis='columns')
df_data_8

In [ ]:
#ONE HOT ENCODING WITH getdummies
df_data_7=df_data_6.copy()
for fname in categorical_cols: 
    dummy_gender = pd.get_dummies(df_data_7[fname], prefix='{}_'.format(fname))
    df_data_7= pd.concat([df_data_7,dummy_gender], axis='columns')

1.2 NORMALIZATION
--

In [ ]:
cMMS=preprocessing.MinMaxScaler()
x = df_data_7['SplitFare'].values.reshape(-1, 1)
x_scaled = cMMS.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled , columns =["SplitFareNorm"]) 
df_data_9= pd.concat([df_data_7, df_scaled ], axis='columns')

1.3 PCA
--

In [ ]:
t0 = time()
X_train = 
n_components = 4
pca = PCA(n_components=n_components, whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

1.4 FEATURE IMPORTANCE
--

In [ ]:
RFC = RandomForestClassifier( n_estimators = 400,
                              max_depth = 10,
                              max_features = "log2",
                              criterion = "gini",
                              min_samples_split = 10,
                              min_impurity_decrease = 0.01,
                              bootstrap = False ,
                              random_state = 2)

RFC.fit(x_train, y_train)
# get importance
importance = RFC.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
dimensions = (14, 8)
fig, ax = plt.subplots(figsize=dimensions)
ax.bar([x for x in range(len(importance))], importance)
plt.xticks(range(x_train.shape[1]), x_train.columns , rotation=90)


2.FILE MANAGEMENT
===

In [ ]:
import pickle
a_dataframe.to_pickle("a_file.pkl")
output = pd.read_pickle("a_file.pkl")

In [ ]:
#La scrittura wb serve perchè pickle scrive in byte non stringhe
f = open("x_train.pkl", "wb")
pickle.dump(x_train, f)
f.close()
pickle_data = (pickle.load(open("x_train.pkl")))
f.close()

In [ ]:
#Save dataframe to feather file
sTempFilePath = './tmp/df_FareResult_4.ftr'
os.makedirs('./tmp',exist_ok=True)
df_FareResult_4.to_feather(sTempFilePath)

#load dataframe from feather file
df_FareResult_4=pd.read_feather(sTempFilePath, columns=None, use_threads=True)

3.PROCESS AND MEMORY MANAGEMENT
===

In [ ]:
del df_data_1
del df_data_2
del df_data_3
del df_data_4
del df_data_5
del df_data_6
del df_data_7
import gc
gc.collect()

4.DATA SHAPE MANAGEMENT
===

In [ ]:
#Add new dimension on array data
np_y_train = y_train[:, np.newaxis]
np_y_train

5.CORRELATION HEATMAPS
===

5.1 Pearson
---

La correlazione di Pearson è utilizzabile con due assunzioni:
<ul>
   <li>la distribuzione dei dati è gaussiana</li>
    <li>la rappresentazione cartesiana è <b>lineare</b></li>
</ul> 
Quindi il primo punto debole di questa correlazione è che un legame non lineare viene interpretato come correlazione 0!
E' il tipo di correlazione più semplice e basato sulla relazione: 

In [ ]:
![PearsonCorr.jpg](attachment:PearsonCorr.jpg)

In [ ]:
df_heatmap = pd.DataFrame(df_data_1.corr(method='pearson'))
mask=np.zeros_like(df_heatmap)
mask[np.triu_indices_from(mask)] = True
f,ax = plt.subplots(figsize=(22,22),facecolor='white')
sns.color_palette("rocket", as_cmap=True)  
sns.heatmap(df_heatmap,annot = True,square=True, linewidths=1.5, cmap='rocket',mask=mask)

5.2 Kendall
---

La correlazione di Kendall invece segue un approccio diverso
 
<ol>
  <li>Ranking: si ordinano i dati secondo la variabile indipendente</li>
  <li>Primo Compare: si confronta la prima variabile indipendente in cima alla tabella ordinata con tutti gli altri sample indipendenti x<sub>i+1</sub> > x<sub>i</sub> </li>
  <li>Secondo Compare: si confronta la prima variabile dipendente associata alla prima variabile indip. con la successiva variabile dipendente verificando se y<sub>i+1</sub> > y<sub>i</sub> </li>
  <li>Concorde o Discorde: se x<sub>i+1</sub> > x<sub>i</sub> e y<sub>i+1</sub> > y<sub>i</sub> sono entrambe vere si incrementa un valore <b>Nc</b> (numero di check concordi)   altrimenti s'incrementa <b>Nd</b> (numero di check discordi) </li> 
  <li>Si eseguono i passi 2 e 3 scorrendo la tabella verso il basso, fissando ogni volta una coppia di variabili dip. e indip. e confrontandole con tutte le altre</li> 
</ol>     

Questo tipo di correlazione è perciò un po' più grossolana perchè non dà una percezione profonda della curva che stiamo analizzando ma il vantaggio è che la relazione potrebbe anche essere<b> non lineare</b>

![KendalCorr.png](attachment:KendalCorr.png)

In [ ]:
df_heatmap = pd.DataFrame(df_data_1.corr(method='kendall'))
mask=np.zeros_like(df_heatmap)
mask[np.triu_indices_from(mask)] = True
f,ax = plt.subplots(figsize=(22,22),facecolor='white')
sns.color_palette("rocket", as_cmap=True)  
sns.heatmap(df_heatmap,annot = True,square=True, linewidths=1.5, cmap='rocket',mask=mask)

5.3 Spearman
---

La correlazione di Spearman è simile a quella di Pearson con la differenza che il sample da confrontare viene ordinato. Successivamente si calcola la distanza tra due set di campioni per mezzo di una semplice differenza portata poi al quadrato. La correlazione di Spearman è molto più sensibile alle variazioni dei dati rispetto alla correlazione di Kendall. Sotto un esempio:

![SpearmanCorr.png](attachment:SpearmanCorr.png)

![SpearmanCorr2.PNG](attachment:SpearmanCorr2.PNG)

In [ ]:
df_heatmap = pd.DataFrame(df_data_1.corr(method='spearman'))
mask=np.zeros_like(df_heatmap)
mask[np.triu_indices_from(mask)] = True
f,ax = plt.subplots(figsize=(22,22),facecolor='white')
sns.color_palette("rocket", as_cmap=True)  
sns.heatmap(df_heatmap,annot = True,square=True, linewidths=1.5, cmap='rocket',mask=mask)

6.STRING MANAGEMENT
===

In [ ]:
for i, Name , Surname in zip( df.index , df["Name"] , df["Surname"]):
    df_index_res = df_data_1.index[(df_data_1["Name"].str.contains(str(Surname))) & (df_data_1["Name"].str.contains(str(Name)))]  
    if len(df_index_res) == 1 and df_index_res != np.nan:
        i_match = df_index_res[0]

In [ ]:
numbers = re.findall('[0-9]+', item)

In [ ]:
 l_index=[index for index in range(len(item)) if item.startswith('.',index) ]  

In [ ]:
dfCabinPositionClass1.at[j,"Name"]=dfCabinPositionClass1["Split1"].iloc[j].split(" ")[1]

7.PLOT FUNCTIONS
===

In [ ]:
def groupplot(data, x , y , hue , n_subf , font_scale , wspace , palette , x_size , y_size ):
    fig = plt.figure( figsize=(x_size, y_size))
    subfigs = fig.subfigures(n_subf, 1, wspace=wspace)
    sns.set(font_scale=font_scale)

    for i in range(data["GroupSize"].max()):
        axs = subfigs[i].subplots(1, 3)
        subfigs[i].suptitle(" GroupSize :"+str(i+1), fontsize='x-large')
        for j , ax in enumerate(axs):
            try:
                sns.barplot(ax=ax, x = x, y= y, hue=hue , palette=palette,
                                    data=data[(data["Pclass"]==j+1) & (data["GroupSize"]==i+1)])
                plt.xticks(rotation=0)
                if j==2:
                    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
                ax.set_title("Class : "+ str(j+1), fontsize=18)

            except ValueError:
                continue


In [ ]:
groupplot( data = df_MeanByGroupAgeDeckNotSurv, x = 'Age Group' , y = 'SplitFare' ,hue = 'CabinDeck' ,
          n_subf= 11 , font_scale = 1.5 , wspace = 3 , palette = palette , x_size = 22, y_size = 100)

In [ ]:
sns.set(font_scale=1.0)
plt.style.use('seaborn-whitegrid')
plt.xticks(rotation=90)
df_data_1['SplitFareBin'] = pd.cut(df_data_1['log10SplitFare'], bins=10)
g = sns.barplot(x = 'SplitFareBin',y = 'Survived', data=df_data_1)

In [ ]:
features = ['Pclass','GroupSize','Age Group','CabinDeck']
def GroupFare (data , features) :
    for i in range(data[features[0]].max()):
        df_Grouped_Count=data.query(features[0]+'=='+str(i+1)).groupby(features[1:4]).count().reset_index()
        df_Grouped_Count[features[0]]=i+1
        df_Grouped_fareMean=data.query(features[0]+'=='+str(i+1)).groupby(features[1:4]).SplitFare.mean().reset_index()
        df_Grouped_fareMean[features[0]]=i+1
        df_Grouped_fareStd=data.query(features[0]+'=='+str(i+1)).groupby(features[1:4]).SplitFare.std().reset_index()
        df_Grouped_fareStd[features[0]]=i+1
        if i==0 :
            df_CountByGroupAgeDeck = df_Grouped_Count.copy() 
            df_MeanByGroupAgeDeck =  df_Grouped_fareMean.copy() 
            df_StdByGroupAgeDeck = df_Grouped_fareStd.copy()
        if i>0 :
            df_CountByGroupAgeDeck = pd.concat([df_CountByGroupAgeDeck, df_Grouped_Count], ignore_index=True)
            df_MeanByGroupAgeDeck = pd.concat([df_MeanByGroupAgeDeck, df_Grouped_fareMean], ignore_index=True)
            df_StdByGroupAgeDeck = pd.concat([df_StdByGroupAgeDeck, df_Grouped_fareStd], ignore_index=True)
    return df_CountByGroupAgeDeck , df_MeanByGroupAgeDeck , df_StdByGroupAgeDeck

df_CountByGroupAgeDeckNotSurv , df_MeanByGroupAgeDeckNotSurv , df_StdByGroupAgeDeckNotSurv = GroupFare(data = df_data_2 , features = features)

In [ ]:
math.isnan(d_row_match['MeanFareUnk']

In [ ]:
np.isnan(d_item['Survived'])

In [ ]:
df_FareResult_4 = df_FareResult_4.append(df3, ignore_index = True)

In [ ]:
df2= df_FareResult_4[l_col_newrow].loc[(df_FareResult_4['CabinDeck']==deck) &
                                              (df_FareResult_4['GroupSize']==1) &
                                              (df_FareResult_4['Pclass']==2) & 
                                              (df_FareResult_4['Age Group']==2) ]

In [ ]:
#Drop rows with unknown Cabin Deck 
df_FareResult_5 = df_FareResult_5.drop(df_FareResult_5 [(df_FareResult_5['CabinDeck']=='Unknown')].index)